In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
import numbers
import re
import os
from functools import reduce

In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/MRSA datasets/')

print(os.getcwd())

Mounted at /content/drive
/content/drive/My Drive/MRSA datasets


# Global variables

In [3]:
dataset_path = 'Datasets'
labels_path = 'Data Labels'
species = {'s.a': 'Staphylococcus aureus',
           'e.c': 'Escherichia coli',
           'a.b': 'Acinetobacter baumannii',
           's.e': 'Salmonella enterica',
           'm.t': 'Mycobacterium tuberculosis',
           'k.p': 'Klebsiella pneumoniae'}

# Staphylococcus aureus
    # PRJNA633996
Chinese_dataset = 'PRJNA633996'
Chinese_labels_file_path = labels_path + '/' + 'PRJNA633996.xlsx'

    # PATRIC_genomes
patric_sa_dataset = 'PATRIC_genomes'
patric_sa_labels_file_path = labels_path + '/' + 'PATRIC_s.aureus.csv'

# Acinetobacter baumannii
    # PRJNA1014981
a_dataset_1 = 'PRJNA1014981'
a_d1_labels_file_path  = labels_path + '/' + 'PRJNA1014981_labels.csv'

# Salmonella enterica
    # PRJNA292666_PRJNA292661
s_dataset = 'PRJNA292666 & PRJNA292661'
s_labels_file_path  = labels_path + '/' + 'PRJNA292666_PRJNA292661_labels.xlsx'

# Mycobacterium tuberculosis
    # PRJNA343736
t_dataset = 'PRJNA343736'
t_labels_file_path  = labels_path + '/' + 'PRJNA343736_labels.csv'

# Escherichia coli
    # PATRIC_e.coli
e_dataset = 'PATRIC_e.coli'
e_labels_file_path = labels_path + '/' + 'PATRIC_e.coli.xlsx'

# Klebsiella pneumoniae
    # PATRIC_klebsiella
k_dataset = 'PATRIC_klebsiella'
k_labels_file_path = labels_path + '/' + 'PATRIC_klebsiella.xlsx'

In [4]:
get_path = lambda df: dataset_path + '/' + df['Species'] + '/' + df['Dataset'] + '/' + df['File Name']
get_name = lambda a: str(a) + '.fna'
# extract_int = lambda a: list(map(int, re.findall(r'\d+', a)))[0]
def extract_float(string):
    a = list(map(float, re.findall(r'\d*\.\d+|\d+', str(string))))
    return a[0] if a else np.nan

# Staphylococcus aureus

## Chinese Dataset

In [5]:
def getChineseLabels(file_path):
    # load labels from excel file
    df = pd.read_excel(Chinese_labels_file_path)
    # add file name and dataset
    df['File Name'] = df['Sample ID'].apply(get_name)
    df['Dataset'] = Chinese_dataset
    df.drop('Sample ID', inplace=True, axis=1)
    # move file name and dataset to be the first column
    temp = df.pop('File Name')
    df.insert(0, 'File Name', temp)
    temp = df.pop('Dataset')
    df.insert(0, 'Dataset', temp)
    df.insert(0, 'Species', species['s.a'])
    # include files that are found on drive only
    df = df[df.apply(get_path, axis=1).apply(os.path.isfile)]
    return df

In [6]:
Chinese_df = getChineseLabels(Chinese_labels_file_path)

## Patric

In [7]:
def get_patric_labels():
    # load labels from excel file
    df = pd.read_csv(patric_sa_labels_file_path)
    df = df.drop(['bacteria_species_general'], axis=1)
    # add filename, species and dataset and drop sample ID
    df['File Name'] = df['genome_id'].apply(get_name)
    df.drop('genome_id', inplace=True, axis=1)
    df['Dataset'] = patric_sa_dataset
    # reorder and add species
    cols = df.columns.tolist()
    new_order = ['Dataset', 'File Name'] + [col for col in cols if col not in ['File Name', 'Dataset']]
    df = df[new_order]
    df.insert(0, 'Species', species['s.a'])
    # map values tp R and S
    for i in df.columns[3:]:
        df[i] = df[i].map({1: 'R', 0: 'S'})
    df = df[df.apply(get_path, axis=1).apply(os.path.isfile)]
    return df

In [8]:
patri_sa_df = get_patric_labels()

# Acinetobacter baumannii

In [9]:
def get_acinetobacter_labels():
    df = pd.read_csv(a_d1_labels_file_path)
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    # add filename, species and dataset and drop sample ID
    df['File Name'] = df['ID'].apply(get_name)
    df['Dataset'] = a_dataset_1

    cols = df.columns.tolist()
    new_order = ['Dataset', 'File Name'] + [col for col in cols if col not in ['File Name', 'Dataset']]
    df = df[new_order]
    df.insert(0, 'Species', species['a.b'])
    df.drop('ID', inplace=True, axis=1)
    df = df.replace({ 'NaN': np.nan})
    df = df[df.apply(get_path, axis=1).apply(os.path.isfile)]
    return df

In [10]:
a_df = get_acinetobacter_labels()

# Salmonella enterica

In [11]:
extract_int = lambda a: list(map(int, re.findall(r'\d+', a)))[0]
drus_map = {'AMP': 'Ampicillin',
            'AUG': 'Amoxicillin-Clavulanic acid',
            'AXO': 'Ceftriaxone',
            'AZI': 'Azithromycin',
            'CHL': 'Chloramphenicol',
            'CIP': 'Ciprofloxacin',
            'COT': 'Trimethoprim-Sulfamethoxazole',
            'FIS': 'Sulfisoxazole',
            'FOX': 'Cefoxitin',
            'GEN': 'Gentamicin',
            'KAN': 'Kanamycin',
            'NAL': 'Nalidixic acid',
            'STR': 'Streptomycin',
            'TET': 'Tetracycline',
            'TIO': 'Ceftiofur'}

MIC_breakpoints = {'Ampicillin': {'R': 32, 'I': 16, 'S': 8},
                   'Amoxicillin-Clavulanic acid': {'R': 32, 'I': 16, 'S': 8},
                   'Ceftriaxone': {'R': 32, 'I': 16, 'S': 8},
                   'Azithromycin': {'R': 32, 'I': np.nan, 'S': 16},
                   'Chloramphenicol': {'R': 32, 'I': 16, 'S': 8},
                   'Ciprofloxacin': {'R': 1, 'I': 0.12, 'S': 0.06},
                   'Trimethoprim-Sulfamethoxazole': {'R': 4, 'I': np.nan, 'S': 2},
                   'Sulfisoxazole': {'R': 512, 'I': np.nan, 'S': 256},
                   'Cefoxitin': {'R': 32, 'I': 16, 'S': 8},
                   'Gentamicin': {'R': 8, 'I': 4, 'S': 2},
                   'Kanamycin': {'R': 64, 'I': 32, 'S': 16},
                   'Nalidixic acid': {'R': 32, 'I': np.nan, 'S': 16},
                   'Streptomycin': {'R': 32, 'I': np.nan, 'S': 16},
                   'Tetracycline': {'R': 16, 'I': 8, 'S': 4},
                   'Ceftiofur': {'R': 8, 'I': 4, 'S': 2}}

def map_MIC(val, map):
    if pd.isna(val):
        return np.nan
    if val <= map['S']:
        return 'S'
    if val >= map['R']:
        return 'R'
    return 'I'

def print_unique_values(df):
    unique_per_column = {col: df[col].unique() for col in df.loc[ : , df.columns != 'SRA Run Accession']}
    for col, uniques in unique_per_column.items():
        print(f"{col: <30}: {uniques}")

In [12]:
def get_salmonella_labels():
    df = pd.read_excel(s_labels_file_path)
    df = df.pivot(index='SRA Run Accession', columns='Antibiotic', values='Laboratory-derived MIC').reset_index()
    df.rename(columns=drus_map, inplace=True)
    df.loc[ : , df.columns != 'SRA Run Accession'] = df.loc[ : , df.columns != 'SRA Run Accession'].map(extract_float)
    for c in df.loc[ : , df.columns != 'SRA Run Accession'].columns:
        df[c] = df[c].apply(lambda x: map_MIC(x, MIC_breakpoints[c]))

    df['File Name'] = df['SRA Run Accession'].apply(get_name)
    df['Dataset'] = s_dataset
    cols = df.columns.tolist()
    new_order = ['Dataset', 'File Name'] + [col for col in cols if col not in ['File Name', 'Dataset']]
    df = df[new_order]
    df.insert(0, 'Species', species['s.e'])
    df.drop('SRA Run Accession', inplace=True, axis=1)
    df = df[df.apply(get_path, axis=1).apply(os.path.isfile)]
    return df

In [13]:
s_df = get_salmonella_labels()

# Escherichia coli

In [14]:
def get_E_coli_labels():
    # load labels from excel file
    df = pd.read_excel(e_labels_file_path)
    df = df.loc[1:]
    df.drop(['Genome Quality', 'Genome Status', 'Contigs',
            'Typing method', 'Testing standard', 'Country',
            'Host', 'Unnamed: 20', 'Total Isolates'], axis = 1, inplace = True)
    # add filename, species and dataset and drop sample ID
    df['File Name'] = df['Genome ID'].apply(get_name)
    df.drop('Genome ID', inplace=True, axis=1)
    df['Dataset'] = e_dataset
    # reorder and add species
    cols = df.columns.tolist()
    new_order = ['Dataset', 'File Name'] + [col for col in cols if col not in ['File Name', 'Dataset']]
    df = df[new_order]
    df.insert(0, 'Species', species['e.c'])
    for i in df.columns[3:]:
        df[i] = df[i].map({'Resistant': 'R', 'Susceptible': 'S', '-': np.nan})
    df = df[df.apply(get_path, axis=1).apply(os.path.isfile)]
    return df

In [15]:
e_df = get_E_coli_labels()

# Mycobacterium tuberculosis

In [16]:
def get_tuberculosis_labels():
    df = pd.read_csv(t_labels_file_path, low_memory = False)
    df = df.drop(['index', 'Isolate', 'run',
                  'run_combined', 'bioproject',
                  'biosample', 'internal', 'path',
                  'Isolate_original', 'category',
                  'PARA_AMINOSALICYLIC_ACID', 'OXIFLOXACIN',
                  'LINEZOLID', 'IMIPENEM', 'ETHIOMIDE'], axis=1)
    df['File Name'] = df['accessions'].apply(get_name)
    df.drop('accessions', inplace=True, axis=1)
    df['Dataset'] = t_dataset

    cols = df.columns.tolist()
    new_order = ['Dataset', 'File Name'] + [col for col in cols if col not in ['File Name', 'Dataset']]
    df = df[new_order]
    df.insert(0, 'Species', species['m.t'])
    df = df[df.apply(get_path, axis=1).apply(os.path.isfile)]
    return df

In [17]:
t_df = get_tuberculosis_labels()

# Klebsiella pneumoniae

In [18]:
def get_patric_labels():
    # load labels from excel file
    df = pd.read_csv(patric_sa_labels_file_path)
    df = df.drop(['bacteria_species_general'], axis=1)
    # add filename, species and dataset and drop sample ID
    df['File Name'] = df['genome_id'].apply(get_name)
    df.drop('genome_id', inplace=True, axis=1)
    df['Dataset'] = patric_sa_dataset
    # reorder and add species
    cols = df.columns.tolist()
    new_order = ['Dataset', 'File Name'] + [col for col in cols if col not in ['File Name', 'Dataset']]
    df = df[new_order]
    df.insert(0, 'Species', species['s.a'])
    # map values tp R and S
    for i in df.columns[3:]:
        df[i] = df[i].map({1: 'R', 0: 'S'})
    df = df[df.apply(get_path, axis=1).apply(os.path.isfile)]
    return df

In [19]:
MIC_breakpoints = {'Piperacillin-Tazobactam': {'R': 32, 'I': 16, 'S': 8},
                   'Ciprofloxacin': {'R': 1, 'I': np.nan, 'S': 0.25},
                   'Gentamicin': {'R': 8, 'I': 4, 'S': 2},
                   'Meropenem': {'R': 4, 'I': np.nan, 'S': 1}}

In [20]:
def get_klebsiella_labels():
    # load labels from excel file
    file = pd.ExcelFile(k_labels_file_path)
    # merge sheets into 1 dataframe and rename columns
    dfs = []
    for drug in file.sheet_names:
      df = pd.read_excel(file, sheet_name = drug)
      df = df.rename(columns={'MIC': drug})
      dfs.append(df)
    df = reduce(lambda left, right: pd.merge(left, right, on='Genome_ID', how='outer'), dfs)
    # map MIC values
    df.columns = [c.split(' ')[0] for c in df.columns]
    for c in df.loc[ : , df.columns != 'Genome_ID'].columns:
          df[c] = df[c].apply(lambda x: map_MIC(x, MIC_breakpoints[c]))
    # add filename, species and dataset and drop sample ID
    df['File Name'] = df['Genome_ID'].apply(get_name)
    df.drop('Genome_ID', inplace=True, axis=1)
    df['Dataset'] = k_dataset
    # reorder and add species
    cols = df.columns.tolist()
    new_order = ['Dataset', 'File Name'] + [col for col in cols if col not in ['File Name', 'Dataset']]
    df = df[new_order]
    df.insert(0, 'Species', species['k.p'])
    df = df[df.apply(get_path, axis=1).apply(os.path.isfile)]
    return df

In [21]:
k_df = get_klebsiella_labels()

# Exporting csv

In [22]:
df = pd.concat([Chinese_df, patri_sa_df, a_df, s_df, t_df, e_df, k_df], ignore_index=True)

In [24]:
col_order = ['Species', 'Dataset', 'File Name',
    'Streptomycin', 'Vancomycin', 'Ceftriaxone',
    'Oxacillin', 'Clindamycin', 'Fusidic acid', 'Erythromycin',
    'Linezolid', 'Methicillin', 'Gentamicin', 'Trimethoprim-Sulfamethoxazole',
    'Ciprofloxacin', 'Tetracycline',
    'Chloramphenicol', 'Penicillin', 'Trimethoprim',
    'Daptomycin', 'Cefoxitin', 'Amikacin', 'Ampicillin-Sulbactam', 'Cefepime',
    'Cefotaxime', 'Ceftazidime', 'Colistin', 'Doripenem', 'Imipenem',
    'Levofloxacin', 'Meropenem', 'Minocycline', 'Moxifloxacin', 'Polymyxin_B',
    'Tigecycline', 'Tobramycin', 'Ampicillin', 'Amoxicillin-Clavulanic acid', 'Azithromycin',
    'Sulfisoxazole', 'Kanamycin', 'Nalidixic acid', 'Ceftiofur', 'Amoxicillin',
    'Capreomycin', 'Clarithromycin', 'Clofazimine', 'Cycloserine', 'Ethambutol',
    'Ethionamide', 'Gatifloxacin', 'Isoniazid', 'Nicotinamide', 'Ofloxacin',
    'para-aminosalicylic acid', 'Prothionamide', 'Pyrazinamide', 'Rifabutin', 'Rifampicin',
    'Aztreonam', 'Cefuroxime', 'Piperacillin-Tazobactam'
]

df = df[col_order]

In [25]:
print(f"{'Species':<30} {'Samples':<10}")
print('-'*40)
for key in species:
    print(f"{species[key]:<30} {len(df[df['Species'] == species[key]]):<8.0f}")
print('-'*40)
print(f"{'Total':<30} {len(df):<10}")

Species                        Samples   
----------------------------------------
Staphylococcus aureus          5929    
Escherichia coli               3112    
Acinetobacter baumannii        859     
Salmonella enterica            5084    
Mycobacterium tuberculosis     3523    
Klebsiella pneumoniae          1556    
----------------------------------------
Total                          20063     


In [ ]:
master_table_dir = 'Tables/Master_Table.csv'
df.to_csv(master_table_dir, index=False, sep=',', header=True, na_rep='N/A')